# Image Generation from Process Models
This notebook can be used to render BPMN diagrams; it is based on the notebook [here](https://github.com/signavio/sap-sam/blob/main/notebooks/3_images_and_XMLs.ipynb)

## Setup
In order to render images, we need to create an account on [academic.signavio.com](academic.signavio.com) or on a commercial SAP Signavio system instance. To setup the integration with your SAP Signavio workspace, we create a file named ``conf.py`` in the `notebooks` directory of this project and specify the following variables:

* ``email``: the email address with which you have registered your account.
* ``pw``: your password -- use a password you do not use elsewhere and do not commit/share the ``conf.py`` file.
* ``tenant_id``: the ID of your workspace, which can be obtained by selecting **Help** - **Workspace information** in the explorer of SAP Signavio Process Manager.
* ``system_instance``: ``'https://academic.signavio.com'`` or the URL of a commercial system instance.

Below, we see an example configuration file:

```
email = 'jane@doe.org'
pw = '1234546'
tenant_id = 'cb0f0496e0ca1234562e881f9f654321'
system_instance = https://academic.signavio.com
```

This allows the underlying code to upload diagrams to a specific folder (named `SAP-SAM`) in your workspace so that the server can generate an image (PNG).
The folder does not need to be created manually (the SAP-SAM API takes care of this).

We import the following dependencies:

In [ ]:
from IPython.display import Image, display
import json
import os
import pandas as pd
import xml.etree.ElementTree as ET
from pathlib import Path
pd.set_option("display.min_rows", 4)
from ImageGenerator import ImageGenerator

from semconstmining.config import Config
from semconstmining.parsing.label_parser.nlp_helper import NlpHelper
from semconstmining.main import get_resource_handler

## Loading Content

In [ ]:
MODEL_COLLECTION = "semantic_sap_sam_filtered"

conf = Config(Path(os.getcwd()).parents[1].resolve(), MODEL_COLLECTION)
nlp_helper = NlpHelper(conf)
resource_handler = get_resource_handler(conf, nlp_helper)

In [ ]:
resource_handler.bpmn_models

In [ ]:
MODEL_ID = "6d39679f96bf44ee90082a4d98aee8f2"

model_info = resource_handler.bpmn_models.loc[MODEL_ID]
model_json = model_info["model_json"]

In addition, we need the model's namespace and name:

In [ ]:
model_namespace = "http://b3mn.org/stencilset/bpmn2.0#"
# If the name cannot be obtained, the diagram can be named manually or by using a dummy label
model_name = json.loads(model_json)['properties']['name']
if model_name == "":
    model_name = MODEL_ID
print('Namespace:', model_namespace)
print('Name:', model_name)

## Rendering Images
Now, we can use the SAP-SAM API to create the diagram in SAP Signavio Process Manager and have it generate the image:

In [ ]:
gen = ImageGenerator()
image_request = gen.generate_image(model_name, model_json, model_namespace)
display(Image(image_request))

Note that the diagram was stored in the `SAP-SAM` folder of the SAP Signavio workspace, but is deleted immediately after the PNG is obtained. In case deletion is undesired, change the method call to `gen.generate_image(model_name, model_json, model_namespace, deletes=False)`.